
###1. Для начала определим широкий топик текста с помощью gensim.

In [ ]:
!pip install nltk

In [ ]:
!pip install spacy

In [ ]:
!pip  install pyLDAvis

In [ ]:
import nltk; nltk.download('stopwords')


!python3 -m spacy download en

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [ ]:
data = df.content.values.tolist()
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]
pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [ ]:
#токенизируем, делаем из каждого предложения список

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

####Тренеруем нашу модель

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=11, 
                                           random_state=100,
                                           update_every=2,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.029*"team" + 0.026*"game" + 0.022*"year" + 0.019*"play" + 0.016*"player" '
  '+ 0.014*"win" + 0.010*"season" + 0.009*"run" + 0.009*"last" + '
  '0.008*"score"'),
 (1,
  '0.162*"ax" + 0.137*"max" + 0.012*"pen" + 0.011*"di_di" + 0.008*"patient" + '
  '0.008*"registration" + 0.008*"inch" + 0.007*"penguin" + 0.007*"leafs" + '
  '0.005*"cub"'),
 (2,
  '0.015*"space" + 0.006*"launch" + 0.006*"engine" + 0.006*"high" + '
  '0.006*"system" + 0.006*"food" + 0.005*"low" + 0.005*"mission" + '
  '0.005*"test" + 0.005*"earth"'),
 (3,
  '0.040*"drive" + 0.025*"car" + 0.011*"bike" + 0.010*"power" + 0.008*"wire" + '
  '0.008*"use" + 0.008*"light" + 0.008*"scsi" + 0.007*"ride" + 0.006*"high"'),
 (4,
  '0.016*"gun" + 0.016*"kill" + 0.010*"attack" + 0.010*"israeli" + '
  '0.010*"child" + 0.010*"war" + 0.010*"drug" + 0.009*"fire" + 0.008*"report" '
  '+ 0.007*"death"'),
 (5,
  '0.008*"say" + 0.007*"people" + 0.007*"slave" + 0.007*"year" + 0.005*"cover" '
  '+ 0.005*"turkish" + 0.005*"leave" + 0.0

###2. Создайте функцию или серию функций, через которую будет удобно подобрать оптимальное число групп - 1 балл за нахождение оптимального числа групп, 1 балл - если это будет не руками, а через функцию

Просто будем считать coherence score для каждого числа групп

In [ ]:
def optimal_q(corpus, id2word, data_lemmatized):
  coh_numTopics = {}
  for n in range(8,30):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=n, 
                                           random_state=100,
                                           update_every=2,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coh = coherence_model_lda.get_coherence()
    coh_numTopics[coh] = n
  return coh_numTopics

In [ ]:
coh_numTopics = optimal_q(corpus, id2word, data_lemmatized)

In [ ]:
max(coh_numTopics.items())

(0.5363387710830442, 11)

Получается, оптимальное число групп - 11

###3. Как вы знаете, gensim считает, что каждый текст содержит несколько топиков, но на следующем этапе вам будет надо создать функцию, которая будет для каждого текста определять один широкий топик, самый главный. Мы предлагаем сделать это так: создайте счётчик, и каждый раз, когда в тексте будет встречаться одно из слов, соответсвующих данной теме, добавляйте к счетчику его вес. 2 балла

In [ ]:
topics = lda_model.show_topics(formatted=False, num_words= 10)

In [ ]:
from collections import Counter

Сделаем все как в инструкции)

In [ ]:
text_topic = {}
id_t = 0
for text in texts:
  id_t +=1
  wide_topics = Counter()
  for w in text:
    for topic in topics:
      for t in topic[1]:
        if t[0] == w:
          wide_topics[topic[0]] += float(t[1])
  text_topic[' '.join(text)] = wide_topics.most_common(1)
#print(text_topic)

Запишем в новый словарь только текст и номер темы, чтобы красиво записать в дата фрейм

In [ ]:
text_topic1 = {}
for text, topic in text_topic.items():
  if topic != []:
    text_topic1[text] = topic[0][0]

In [ ]:
colNames = ['topic']
df = pd.DataFrame.from_dict(text_topic1, orient='index', columns=colNames)

In [ ]:
print(df)

                                                    topic
where thing car nntp_poste host park line wonde...      3
poll final call summary final call clock report...      9
engineering computer network distribution_usa l...     10
division line host write write article know chi...     10
question distribution article write clear cauti...     10
...                                                   ...
scan city reply line consultation cheap also we...     10
screen medford old problem screen blank sometim...      3
este mount mail group line instal try mount coo...      9
line article write boy embarasse trivial faq gi...     10
steal organization line distribution_usa host s...      9

[11279 rows x 1 columns]


Запишем его в файл

In [ ]:
df.to_csv('text-topic.csv')

###4. После того, как у вас получится какое-то количество групп (наборов текстов с общим топиком). Внутри каждой из этих групп посчитайте тф_идф для каждого текста (Т. е. возьмите все тексты с одинаковой темой за ваш корпус и посчитайте для каждого из этих текстов тф_идф). 3 балла

Теперь вывернем наш словарь наизнанку

In [ ]:
topic_text1 = collections.defaultdict(list)
for t_t in text_topic.items():
  if t_t[1] != []:
    topic_text1[t_t[1][0][0]].append(t_t[0]) #топик:[тексты]

In [ ]:
topic_text = dict(topic_text1)

Для подсчета tf idf и нахождения топ 5 слов с самым высоким значением возьмем код с семинара

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

И пройдемся им по всем топикам

In [ ]:
top_words = {}
for i in topic_text.keys():
  vectors = vectorizer.fit_transform(topic_text[i])
  feature_names = np.array(vectorizer.get_feature_names())
  for j in range(len(topic_text[i])):
    article_vector = vectors[j, :]
    words = get_top_tf_idf_words(article_vector, feature_names, 5)
    top_words[topic_text[i][j]] = words.tolist()

Сделаем df и запишем его в файл

In [ ]:
tw = pd.DataFrame.from_dict(top_words, orient='index')

In [ ]:
tw.head()

,0,1,2,3,4
where thing car nntp_poste host park line wonder could enlighten car see day door sport car look late early call bricklin door really small addition separate rest body know tellme model name engine year production car make history info funky look car mail thank bring neighborhood lerxst,car,door,where,funky,tellme
insurance rate performance car line latech newsreader_nnr recently post article ask kind rate single male driver yrs old pay performance car here summary reply receive be anymore close enough twin turbo model ticket accident take defensive driving airbag security single year state farm insurance include additional umbrella policy car house base policy standard policy require defensive drive course less buy car company never accident ticket year quote hope help remember name correctly ask insurance performance car well last year similar situation buy car make inquiry age car drive record turn may insurance go mos also be single incur high rate company have get couple friend pay different in company also maybe be lucky hope info help group be thunderbird sc never make claim insurance though hit several time negligent driver could see stop sign fiddle radio move violation last month go failure clear intersection still say damn light yellow one go go record rate state passive restraint deduction liability deductible comprehensive deductible collision roughly year pay year disclaimer be engineer play work hell thing kill man take away s ever go age group experience may interesting own decide buy gift exotic car front runner include model year narrow like simplicity handling snob appeal drive turbo less money feature performance almost personal luxury car well acceleration high top speed almost ready give buying impulse decide stop insurance agent office way ask would happen rate car buy rate consider year rate safe car slight increase car year new low risk division continue handle account buy change standard high rate company rate double go story well cover rest year much faster actually fast standard make sense book say insure corvette reason underwriter consider supra driver traditional conservative eventually go number reason porsche dealer nice salesman get interested tough high pressure guy back room equal monthly payment would take year longer pay porsche high insurance conclude high insurance relate probability auto theft everyone entitle opinion imagination important knowledge live idaho many year buy insurance year turn immediately drop year accident strictly age change rate stay pretty much sell car pickup year less real amazing thing wake age feel much responsible information single move violation year let see be single male clean driving record pay year good deal ask think get talon think insurance high turbo sport car clean record small town around year age nearby city rate higher have get insure protege year year old state insurance state farm info car co insurance yearly insurance age date license mountain_view move violation hope help post summary possible vijay email single year old turbo full cover reasonable liability ticket violation accident knock wood mass thing make huge difference mass town live be personally good town reasonable distance move best would go move bad would also accident couple ticket would probably add year old ticket go record year full coverage state farm get small discount alarm system year live actually live city price would year be case be interested be insure month s personal total property deductible glass towing state farm drive less year think seriously rip performance car list record clean pay try call insurance dealer could find rate suppose standardize have find place initially call give ridiculously high hit much low also change insurance_companie rate go renewal accident ticket car get old maintain low rate always careful come insurance_companies serge,year,insurance,rate,car,age
scsi problem line know specific problem mention message definitely scsi problem

In [ ]:
tw.to_csv('text_top5.csv')